# training demo for pytorch models

In [1]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.metrics import r2_score

In [2]:
# load training data
Xtr_loadpath = 'Xtr.csv'
Xts_loadpath = 'Xts.csv'
ytr_loadpath = 'ytr.csv'

Xtr = np.loadtxt(Xtr_loadpath, delimiter=",")
Xts = np.loadtxt(Xts_loadpath, delimiter=",")
ytr = np.loadtxt(ytr_loadpath, delimiter=",")

In [3]:
# standardize the training data
Xtr_standardized = 2.*Xtr # revise this line as needed
Xts_standardized = 2.*Xts # revise this line as needed

# save the standardized training data
Xtr_savepath = 'Xtr_pytorch.csv'
Xts_savepath = 'Xts_pytorch.csv'
ytr_savepath = 'ytr_pytorch.csv'
yts_hat_savepath = 'yts_hat_pytorch.csv'

np.savetxt(Xtr_savepath, Xtr_standardized, delimiter=",")
np.savetxt(Xts_savepath, Xts_standardized, delimiter=",")
np.savetxt(ytr_savepath, ytr, delimiter=",")

In [4]:
# create a model
d_in = Xtr.shape[1]
d_out = 1

class DumbNet(nn.Module):
    def __init__(self):
        super(DumbNet, self).__init__()
        self.Dense = nn.Linear(d_in,d_out)
    def forward(self,x):
        out = self.Dense(x)
        return out

model = DumbNet()   

# Usually, we would train the model at this point. 
# But this is only a demo, so we'll use the randomly initialized weights.

In [5]:
# compute the training accuracy
with torch.no_grad():
    predict = model(torch.Tensor(Xtr_standardized)).detach().numpy().ravel()

r2 = r2_score(ytr,predict)
print('training R2: ',r2)

training R2:  -10805.060341433771


In [6]:
# save the model: you must use the .pth format for pytorch models!
model_savepath = 'model.pth'

# To save a PyTorch model, we first pass an input through the model, 
# and then save the "trace". 
# For this purpose, we can use any input. 
# We will create a random input with the proper dimension.
x = torch.randn(d_in) # random input
x = x[None,:] # add singleton batch index
with torch.no_grad():
    traced_cell = torch.jit.trace(model, (x))

# Now we save the trace
torch.jit.save(traced_cell, model_savepath)

In [7]:
# generate kaggle submission file using the validation script
!python {"validation.py " + model_savepath + " --Xts_path " + Xts_savepath + " --Xtr_path " + Xtr_savepath + " --yts_hat_path " + yts_hat_savepath } 

training R2 =  -10805.060341433771
test target predictions saved in yts_hat_pytorch.csv
